# Advanced Querying Mongo

Importing libraries and setting up connection

In [5]:
%pip install pymongo

from pymongo import MongoClient
client = MongoClient()
str_conn = 'mongodb://localhost:27017'
cursor = MongoClient(str_conn)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
db = client.Companies
colec = db.tabla_1

In [7]:
client.list_database_names()

['Companies', 'admin', 'config', 'local']

In [9]:
#list(colec.find())

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [19]:
query = {'name': 'Babelgum'}

select = {'name' : True, '_id' : False}

list(colec.find(query, select))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [20]:
query = {'number_of_employees': {'$gte': 5000}}

select = {'name' : True, 'number_of_employees' : True, '_id' : False}

list(colec.find(query, select).sort('number_of_employees', -1).limit(20))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [35]:
query = {'$and': [{'founded_year': {'$lte': 2005}},
                {'founded_year': {'$gte': 2000}}]}

select = {'name' : True, 'founded_year' : True, '_id' : False}

list(colec.find(query, select).limit(20))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [26]:
query = {'$and': [{'ipo.valuation_amount': {'$gte': 1e8}},
                  {'founded_year': {'$lte': 2010}}]}

select = {'name' : True, 'ipo.valuation_amount' : True, '_id' : False}

list(colec.find(query, select).sort('ipo.valuation_amount', -1).limit(20))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000.0}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000.0}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000.0}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000.0}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000.0}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000.0}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000.0}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000.0}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [34]:
query = {'$and': [{'number_of_employees': {'$lte': 1e3}},
                {'founded_year': {'$lte': 2005}}]}

select = {'name' : True, 'founded_year' : True, 'number_of_employees' : True, '_id' : False}

list(colec.find(query, select).limit(10))

[{'name': 'Wetpaint', 'number_of_employees': 47, 'founded_year': 2005},
 {'name': 'AdventNet', 'number_of_employees': 600, 'founded_year': 1996},
 {'name': 'Digg', 'number_of_employees': 60, 'founded_year': 2004},
 {'name': 'Fox Interactive Media',
  'number_of_employees': 0,
  'founded_year': 1979},
 {'name': 'Plaxo', 'number_of_employees': 50, 'founded_year': 2002},
 {'name': 'Technorati', 'number_of_employees': 35, 'founded_year': 2002},
 {'name': 'AddThis', 'number_of_employees': 120, 'founded_year': 2004},
 {'name': 'Jingle Networks', 'number_of_employees': 35, 'founded_year': 2005},
 {'name': 'Meetup', 'number_of_employees': 75, 'founded_year': 2002},
 {'name': 'LifeLock', 'number_of_employees': 644, 'founded_year': 2005}]

### 6. All the companies that don't include the `partners` field.

In [30]:
query = {'partners': {'$exists': True}}

select = {'name' : True, '_id' : False}

list(colec.find(query, select).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [31]:
query = {'category_code': None}

select = {'name' : True, '_id' : False}

list(colec.find(query, select).limit(10))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [33]:
query = {'$and': [{'number_of_employees': {'$gte': 1e2}},
                {'number_of_employees': {'$lte': 1e3}}]}

select = {'name' : True, 'number_of_employees' : True, '_id' : False}

list(colec.find(query, select).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [54]:
select = {'name' : True, 'ipo.valuation_amount' : True, '_id' : False}

list(colec.find({}, select).sort('ipo.valuation_amount', -1).limit(20))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000.0}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000.0}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000.0}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000.0}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000.0}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000.0}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000.0}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000.0}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [55]:
select = {'name' : True, 'number_of_employees' : True, '_id' : False}

list(colec.find({}, select).sort('number_of_employees', -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [59]:
query = {'founded_month': {'$gte': 6}}

select = {'name' : True, 'founded_month' : True, '_id' : False}

#list(colec.find(query, select).limit(1000))

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [63]:
query = {'$and': [{'founded_year': {'$lte': 2000}},
                {'acquisition.price_amount': {'$gte': 1e7}}]}

select = {'name' : True, 'founded_year' : True, 'acquisition.price_amount' : True, '_id' : False}

list(colec.find(query, select).limit(10))

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets',
  'founded_year': 1999,
  'acquisition': {'price_amount': 160000000}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'Zappos',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Sabre',
  'founded_year': 1960,
  'acquisition': {'price_amount': 4300000000.0}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [67]:
query = {'$and': [{'acquisition.acquired_year': {'$gte': 2010}}]}

select = {'name' : True, 'acquisition' : True, '_id' : False}

#list(colec.find(query, select).sort('acquisition', -1).limit(10))

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [69]:
select = {'name' : True, 'founded_year' : True, '_id' : False}

list(colec.find({}, select).sort('founded_year', -1).limit(20))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Springleap', 'founded_year': 2012},
 {'name': 'Skydeck', 'founded_year': 2012},
 {'name': 'Mobiluck', 'founded_year': 2012},
 {'name': 'headr', 'founded_year': 2012},
 {'name': 'FoodCare', 'founded_year': 2012},
 {'name': 'Bling Easy', 'founded_year': 2012},
 {'name': 'Jumbuck Entertainment', 'founded_year': 2012}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [72]:
query = {'founded_day': {'$lte': 7}}

select = {'name' : True, 'founded_day' : True, 'acquisition.price_amount' : True, '_id' : False}

list(colec.find(query, select).sort('acquisition.price_amount', -1).limit(10))

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense',
  'founded_day': 1,
  'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee',
  'founded_day': 1,
  'acquisition': {'price_amount': 321000000}},
 {'name': 'BlueLithium',
  'founded_day': 1,
  'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [73]:
query = {'$and': [{'category_code': 'web'},
                {'number_of_employees': {'$gte': 4e3}}]}


select = {'name' : True, 'founded_day' : True, 'number_of_employees' : True, '_id' : False}

list(colec.find(query, select).sort('number_of_employees', 1).limit(10))

[{'name': 'Expedia', 'number_of_employees': 4400, 'founded_day': None},
 {'name': 'AOL', 'number_of_employees': 8000, 'founded_day': 24},
 {'name': 'Webkinz', 'number_of_employees': 8657, 'founded_day': 25},
 {'name': 'Rakuten', 'number_of_employees': 10000, 'founded_day': 7},
 {'name': 'Los Angeles Times Media Group',
  'number_of_employees': 10000,
  'founded_day': 4},
 {'name': 'Groupon', 'number_of_employees': 10000, 'founded_day': 11},
 {'name': 'Yahoo!', 'number_of_employees': 13600, 'founded_day': 1},
 {'name': 'eBay', 'number_of_employees': 15000, 'founded_day': 1},
 {'name': 'Experian', 'number_of_employees': 15500, 'founded_day': None}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [77]:
query = {'$and': [{'price_currency_code': 'EUR'},
                {'acquisition.price_amount': {'$gte': 1e7}}]}

select = {'name' : True, 'acquisition.price_amount' : True, 'price_currency_code' : True, '_id' : False}

list(colec.find(query, select).limit(10))

[]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [79]:
query = {'acquisition.acquired_month': {'$lte': 3}}

select = {'name' : True, 'acquisition.acquired_month' : True, '_id' : False}

list(colec.find(query, select).limit(10))

[{'name': 'Kyte', 'acquisition': {'acquired_month': 1}},
 {'name': 'NetRatings', 'acquisition': {'acquired_month': 2}},
 {'name': 'blogTV', 'acquisition': {'acquired_month': 3}},
 {'name': 'Livestream', 'acquisition': {'acquired_month': 1}},
 {'name': 'iContact', 'acquisition': {'acquired_month': 2}},
 {'name': 'Coghead', 'acquisition': {'acquired_month': 2}},
 {'name': 'Dailymotion', 'acquisition': {'acquired_month': 2}},
 {'name': 'Netvibes', 'acquisition': {'acquired_month': 2}},
 {'name': 'Flickr', 'acquisition': {'acquired_month': 3}},
 {'name': 'BabyCenter', 'acquisition': {'acquired_month': 3}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code